In [3]:
import kinase_library as kl
import numpy as np
import pandas as pd

In [8]:
prot_info = pd.read_json('../data/phosphosite_sequences/phosphosite_df.json')
prot_info

,id,sites,sequence
0,A0A024R4G9,"[14, 16, 20]",MTVLEAVLEIQAITGSRLLSMVPGPARPPGSCWDPTQCTRTWLLSH...
1,A0A075B759,"[40, 79, 93, 119]",MVNSVVFFEITRDGKPLGRISIKLFADKIPKTAENFRALSTGEKGF...
2,A0A087WP46,"[359, 972, 973, 974, 988, 997, 1000, 1005, 101...",MARDGAEQPDSGPLPRPSPCPQEDRASNLMPPKPPRTWGLQLQGPS...
3,A0A087WPF7,"[32, 43, 622, 626, 798, 941, 956, 1031, 1038, ...",MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...
4,A0A087WQ53,[58],MGQNNNVTEFILLGLTQDPAGQKVLFVMFLLIYIVKIVGNLLIVGT...
...,...,...,...
42252,XP_997087,"[347, 907, 915, 918, 927]",MENFLALMNSISDTWMSPSCMDIAMDMGIAFVCGAGLFFLLLPFLK...
42253,YP_009725299,"[504, 660, 661, 794, 1826]",APTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVEL...
42254,YP_009725305,[5],NNELSPVALRQMSCAAGTTQTACTDDNALAYYNTTKGGRFVLALLS...
42255,YP_009725309,[56],AENVTGLFKDCSKVITGLHPTQAPTHLSVDTKFKTEGLCVDIPGIP...


In [83]:
kinase_info = pd.read_csv('../data/kinase_library/kinome_information.tsv', sep='\t')
kinase_info

,MATRIX_NAME,KINASE,GENE_NAME,TYPE,SUBTYPE,FAMILY,UNIPROT_ID,UNIPROT_ENTRY_NAME,PDB_ID,KL_LIBRARY,DUAL_SPECIFICITY,DISPLAY_NAME
0,AAK1,AAK1,AAK1,ser_thr,STK,Other,Q2M2I8,AAK1_HUMAN,4WSQ,ser_thr,False,AAK1
1,ACVR2A,ACVR2A,ACVR2A,ser_thr,STK,TKL,P27037,AVR2A_HUMAN,3Q4T,ser_thr,False,ACVR2A
2,ACVR2B,ACVR2B,ACVR2B,ser_thr,STK,TKL,Q13705,AVR2B_HUMAN,2H62,ser_thr,False,ACVR2B
3,AKT1,AKT1,AKT1,ser_thr,STK,AGC,P31749,AKT1_HUMAN,1H10,ser_thr,False,AKT1
4,AKT2,AKT2,AKT2,ser_thr,STK,AGC,P31751,AKT2_HUMAN,1GZK,ser_thr,False,AKT2
...,...,...,...,...,...,...,...,...,...,...,...,...
397,VEGFR2,VEGFR2,KDR,tyrosine,RTK,VEGFR,P35968,VGFR2_HUMAN,1VR2,tyr,False,VEGFR2
398,VEGFR3,VEGFR3,FLT4,tyrosine,RTK,VEGFR,P35916,VGFR3_HUMAN,4BSJ,tyr,False,VEGFR3
399,WEE1_TYR,WEE1,WEE1,tyrosine,ncTK,WEE,P30291,WEE1_HUMAN,1X8B,ser_thr_tyr,False,WEE1 (Tyr)
400,YES,YES,YES1,tyrosine,nRTK,SRC,P07947,YES_HUMAN,2HDA,tyr,False,YES


In [84]:
kinase_info=kinase_info.set_index('MATRIX_NAME')

In [85]:
def mark_fn(row):
    buf = []
    split_seq = list(row['sequence'])
    for i in row['sites']:
        idx = eval(i) - 1
        split_seq[idx] = str.lower(split_seq[idx])
    
    for i in row['sites']:
        idx = eval(i) - 1
        left = idx - 6
        left_pad = []
        while left < 0:
            left_pad.append('_')
            left += 1

        right = idx + 7
        right_pad = []
        while right > len(split_seq):
            right_pad.append('_')
            right -= 1
        window = split_seq[left:right]
        # window[idx] = window[idx] + '*'
        buf.append("".join(left_pad + window + right_pad))

    preds = {
        'Name' : [],
        'Family' : [],
        'Score' : [],
        'Score Rank' : [],
        'Percentile' : [],
        'Percentile Rank' : []
    }
    for window in buf:
        sub = kl.Substrate(window, pp=True) # using phosphopriming
        prediction = sub.predict().reset_index(names='Name').iloc[0].to_dict()
        preds['Family'].append(kinase_info.loc[prediction['Name']]['FAMILY'])
        for k, v in prediction.items():
            preds[k].append(v)

    return preds

In [81]:
mark_fn(prot_info.iloc[0])

{'Name': ['GAK', 'GSK3B', 'IKKA'],
 'Family': ['Other', 'CMGC', 'Other'],
 'Score': [3.4399, 5.8656, 3.8763],
 'Score Rank': [1, 1, 1],
 'Percentile': [99.95, 100.0, 99.98],
 'Percentile Rank': [1, 1, 1]}

In [ ]:
test_preds = prot_info[:5].apply(mark_fn, axis=1)

In [59]:
test_df = prot_info[:5]

In [69]:
test_df['kinase_preds'] = test_preds
test_df

C:\Users\Samo\AppData\Local\Temp\ipykernel_22580\2655177893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['kinase_preds'] = test_preds


,id,sites,sequence,kinase_preds
0,A0A024R4G9,"[14, 16, 20]",MTVLEAVLEIQAITGSRLLSMVPGPARPPGSCWDPTQCTRTWLLSH...,"{'Name': ['GAK', 'GSK3B', 'IKKA'], 'Score': [3..."
1,A0A075B759,"[40, 79, 93, 119]",MVNSVVFFEITRDGKPLGRISIKLFADKIPKTAENFRALSTGEKGF...,"{'Name': ['MAPKAPK5', 'FGR', 'BIKE', 'ALPHAK3'..."
2,A0A087WP46,"[359, 972, 973, 974, 988, 997, 1000, 1005, 101...",MARDGAEQPDSGPLPRPSPCPQEDRASNLMPPKPPRTWGLQLQGPS...,"{'Name': ['PRP4', 'YANK3', 'FAK', 'PRPK', 'BUB..."
3,A0A087WPF7,"[32, 43, 622, 626, 798, 941, 956, 1031, 1038, ...",MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,"{'Name': ['LKB1', 'HASPIN', 'GSK3B', 'NEK11', ..."
4,A0A087WQ53,[58],MGQNNNVTEFILLGLTQDPAGQKVLFVMFLLIYIVKIVGNLLIVGT...,"{'Name': ['PDGFRA'], 'Score': [2.9438], 'Score..."


In [70]:
test_df.to_json('ki_test.json')

In [71]:
pd.read_json('ki_test.json')

,id,sites,sequence,kinase_preds
0,A0A024R4G9,"[14, 16, 20]",MTVLEAVLEIQAITGSRLLSMVPGPARPPGSCWDPTQCTRTWLLSH...,"{'Name': ['GAK', 'GSK3B', 'IKKA'], 'Score': [3..."
1,A0A075B759,"[40, 79, 93, 119]",MVNSVVFFEITRDGKPLGRISIKLFADKIPKTAENFRALSTGEKGF...,"{'Name': ['MAPKAPK5', 'FGR', 'BIKE', 'ALPHAK3'..."
2,A0A087WP46,"[359, 972, 973, 974, 988, 997, 1000, 1005, 101...",MARDGAEQPDSGPLPRPSPCPQEDRASNLMPPKPPRTWGLQLQGPS...,"{'Name': ['PRP4', 'YANK3', 'FAK', 'PRPK', 'BUB..."
3,A0A087WPF7,"[32, 43, 622, 626, 798, 941, 956, 1031, 1038, ...",MDGPTRGHGLRKKRRSRSQRDRERRSRAGLGTGAAGGIGAGRTRAP...,"{'Name': ['LKB1', 'HASPIN', 'GSK3B', 'NEK11', ..."
4,A0A087WQ53,[58],MGQNNNVTEFILLGLTQDPAGQKVLFVMFLLIYIVKIVGNLLIVGT...,"{'Name': ['PDGFRA'], 'Score': [2.9438], 'Score..."


In [87]:
preds = prot_info.apply(mark_fn, axis=1)

KeyError: 'CDKL2'

In [ ]:
prot_info['kinase_preds'] = preds
prot_info.to_json('prots_with_kinase_preds.json')

In [ ]:
import json

with open('../data/preds/encoder_S_60_focal_preds_annotated.pt', 'r') as f:
    annotated = json.load(f)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, average_precision_score

for prot in annotated.keys():
    length = len(prot_info.loc[prot]['sequence'])


In [91]:
import torch

unanotated = torch.load('../data/preds/encoder_S_60_focal_preds.pt', map_location='cpu')

TypeError: tuple.count() takes exactly one argument (0 given)